# Dataset Preparation 

In [6]:

import csv
import pandas as pd

#load data
AAPL_path = "./AAPL.csv"

df = pd.read_csv(AAPL_path)


print(AAPL_path)
print(df.head())




./AAPL.csv
         Date      Open      High       Low     Close  Adj Close       Volume
0  1980-12-12  0.513393  0.515625  0.513393  0.513393   0.023186  117258400.0
1  1980-12-15  0.488839  0.488839  0.486607  0.486607   0.021977   43971200.0
2  1980-12-16  0.453125  0.453125  0.450893  0.450893   0.020364   26432000.0
3  1980-12-17  0.462054  0.464286  0.462054  0.462054   0.020868   21610400.0
4  1980-12-18  0.475446  0.477679  0.475446  0.475446   0.021473   18362400.0


In [13]:
# filter out dropping Date and Adj Close 

df_filtered = df.drop(columns=['Date', 'Adj Close'])
print(df_filtered.head())

       Open      High       Low     Close       Volume
0  0.513393  0.515625  0.513393  0.513393  117258400.0
1  0.488839  0.488839  0.486607  0.486607   43971200.0
2  0.453125  0.453125  0.450893  0.450893   26432000.0
3  0.462054  0.464286  0.462054  0.462054   21610400.0
4  0.475446  0.477679  0.475446  0.475446   18362400.0


## 70% Training datat and 30 % test Date

In [22]:
from sklearn.model_selection import train_test_split

X_data = df_filtered[['Open', 'High', 'Low', 'Volume']]

# Y_data represents the stock closing price
Y_data = df_filtered['Close']

# split the date into train 70% and test 30 %
X_data_train, X_data_test, Y_data_train, Y_data_test = train_test_split(X_data, Y_data, test_size = 0.3, random_state = 42) 

print("X_date_train shape:", X_data_train.shape)
print("X_data_test shape: ", X_data_test.shape)
print("Y_data_train shape: ", Y_data_train.shape)
print("Y_data_test shape: ", Y_data_test.shape)


X_date_train shape: (6689, 4)
X_data_test shape:  (2867, 4)
Y_data_train shape:  (6689,)
Y_data_test shape:  (2867,)


## Normalizing the input features

In [38]:
from scipy.stats import zscore

df = pd.read_csv(AAPL_path, na_values=['NA', "?"])

z_columns = ['Open', 'High', 'Low', 'Volume']
#remove rows with NaN values
df.dropna(inplace =True)

df[z_columns] = df[z_columns].apply(zscore)

print(df.head())



         Date      Open      High       Low     Close  Adj Close    Volume
0  1980-12-12 -0.547694 -0.548386 -0.546699  0.513393   0.023186  0.339544
1  1980-12-15 -0.548228 -0.548964 -0.547287  0.486607   0.021977 -0.503910
2  1980-12-16 -0.549006 -0.549735 -0.548071  0.450893   0.020364 -0.705767
3  1980-12-17 -0.548811 -0.549494 -0.547826  0.462054   0.020868 -0.761258
4  1980-12-18 -0.548520 -0.549205 -0.547532  0.475446   0.021473 -0.798639
